# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/Users/albertdavis/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    #TODO 1: Add your code here
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_eng.columns = ["Title", "Year", "Synopsis", "Review"]
    df_fr.columns = ["Title", "Year", "Synopsis", "Review"]
    df_sp.columns = ["Title", "Year", "Synopsis", "Review"]
    df_eng["Original Language"] = "en"
    df_fr["Original Language"] = "fr"
    df_sp["Original Language"] = "sp"
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    return df

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",sp
23,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",sp
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en
26,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",sp
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en
20,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",sp
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""Je ne peux pas croire que j'ai perdu du temps...",fr
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0])
    return decoded

/Users/albertdavis/anaconda3/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df[df["Original Language"] == "fr"]["Review"]
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df[df["Original Language"] == "fr"]["Synopsis"]
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[df["Original Language"] == "sp"]["Review"]
es_reviews_en = es_reviews.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[df["Original Language"] == "sp"]["Synopsis"]
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
fr_rows = df["Original Language"] == "fr"
sp_rows = df["Original Language"] == "sp"
df.loc[fr_rows, "Review"] = fr_reviews_en.values
df.loc[sp_rows, "Review"] = es_reviews_en.values
df.loc[fr_rows, "Synopsis"] = fr_synopsis_en.values
df.loc[sp_rows, "Synopsis"] = es_synopsis_en.values

In [6]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
18,Les Visiteurs en Amérique,2000,<pad> In this continuation of the French comed...,"<pad> ""The film is a total waste of time. The ...",fr
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en
20,Roma,2018,<pad> Cleo (Yalitza Aparicio) is a young domes...,"<pad> ""Rome is a beautiful and moving film tha...",sp
25,Águila Roja,(2009-2016),<pad> This Spanish television series follows t...,"<pad> ""Red Eagle is a boring and uninteresting...",sp
29,El Incidente,2014,"<pad> In this Mexican horror film, a group of ...","<pad> ""The Incident is a boring and frightless...",sp
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
26,Toc Toc,2017,"<pad> In this Spanish comedy, a group of peopl...","<pad> ""Toc Toc is a boring and unoriginal film...",sp
13,Les Choristes,2004,<pad> This film tells the story of a music tea...,"<pad> ""The Choristes are a beautiful film that...",fr
15,Le Dîner de Cons,1998,<pad> The film follows the story of a group of...,"<pad> ""I didn't like this movie at all. The co...",fr
16,La Tour Montparnasse Infernale,2001,<pad> Two incompetent office workers find them...,"<pad> ""I can't believe I've wasted time watchi...",fr


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    prediction = classifier(text)[0]
    print(prediction)
    return prediction["label"]

In [8]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df = df.dropna()
df["Sentiment"] = df["Review"].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

{'label': 'POSITIVE', 'score': 0.9998880624771118}
{'label': 'POSITIVE', 'score': 0.9998817443847656}
{'label': 'POSITIVE', 'score': 0.999884843826294}
{'label': 'POSITIVE', 'score': 0.999883770942688}
{'label': 'POSITIVE', 'score': 0.999885082244873}
{'label': 'NEGATIVE', 'score': 0.9996751546859741}
{'label': 'NEGATIVE', 'score': 0.9998021721839905}
{'label': 'NEGATIVE', 'score': 0.9998024106025696}
{'label': 'NEGATIVE', 'score': 0.9997988343238831}
{'label': 'NEGATIVE', 'score': 0.999816358089447}
{'label': 'POSITIVE', 'score': 0.9998891353607178}
{'label': 'POSITIVE', 'score': 0.99986732006073}
{'label': 'POSITIVE', 'score': 0.9998892545700073}
{'label': 'POSITIVE', 'score': 0.9998902082443237}
{'label': 'POSITIVE', 'score': 0.9998903274536133}
{'label': 'NEGATIVE', 'score': 0.9997155070304871}
{'label': 'NEGATIVE', 'score': 0.9998207688331604}
{'label': 'NEGATIVE', 'score': 0.9998095631599426}
{'label': 'NEGATIVE', 'score': 0.9998236298561096}
{'label': 'NEGATIVE', 'score': 0.9998

In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
27,El Bar,2017,<pad> A group of people are trapped in a bar a...,"<pad> ""The Bar is a ridiculous and meaningless...",sp,NEGATIVE
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en,POSITIVE
21,La Casa de Papel,(2017-2021),<pad> This Spanish television series follows a...,"<pad> ""The Paper House is an exciting and addi...",sp,POSITIVE
14,Le Fabuleux Destin d'Amélie Poulain,2001,<pad> This romantic comedy tells the story of ...,"<pad> ""The Fabulous Destiny of Amélie Poulain ...",fr,POSITIVE
26,Toc Toc,2017,"<pad> In this Spanish comedy, a group of peopl...","<pad> ""Toc Toc is a boring and unoriginal film...",sp,NEGATIVE
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en,POSITIVE
16,La Tour Montparnasse Infernale,2001,<pad> Two incompetent office workers find them...,"<pad> ""I can't believe I've wasted time watchi...",fr,NEGATIVE
20,Roma,2018,<pad> Cleo (Yalitza Aparicio) is a young domes...,"<pad> ""Rome is a beautiful and moving film tha...",sp,POSITIVE
15,Le Dîner de Cons,1998,<pad> The film follows the story of a group of...,"<pad> ""I didn't like this movie at all. The co...",fr,NEGATIVE
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en,NEGATIVE


In [10]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)